In [9]:
import os
from dotenv import load_dotenv
from google.cloud import vision
import pandas as pd

# Carregar variáveis do .env
load_dotenv()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

def extrair_texto_puro(image_path):
    """
    Extrai o texto puro de uma imagem usando o Google Cloud Vision.

    Args:
        image_path (str): Caminho da imagem local.

    Returns:
        str: Texto extraído da imagem.
    """
    # Configurar cliente
    client = vision.ImageAnnotatorClient()

    # Carregar a imagem
    with open(image_path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)

    # Processar a imagem com Google Cloud Vision
    response = client.text_detection(image=image)
    texts = response.text_annotations

    if not texts:
        return "Nenhum texto detectado."

    # Retornar o texto completo
    return texts[0].description

def processar_diretorio_imgs(diretorio):
    """
    Lê todas as imagens de um diretório, transcreve o texto e cria um DataFrame.

    Args:
        diretorio (str): Caminho do diretório com as imagens.

    Returns:
        pd.DataFrame: DataFrame contendo o caminho da imagem e o texto extraído.
    """
    # Lista para armazenar os resultados
    resultados = []

    # Percorrer todos os arquivos no diretório
    for arquivo in os.listdir(diretorio):
        caminho_completo = os.path.join(diretorio, arquivo)

        # Verificar se é um arquivo de imagem
        if arquivo.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            print(f"Processando imagem: {caminho_completo}")
            texto = extrair_texto_puro(caminho_completo)
            resultados.append({"arquivo": caminho_completo, "transcricao": texto})

    # Criar DataFrame
    return pd.DataFrame(resultados)

In [10]:
if __name__ == "__main__":
    # Caminho para o diretório de imagens
    diretorio_imgs = "imgs"

    # Processar as imagens e criar o DataFrame
    df = processar_diretorio_imgs(diretorio_imgs)

    # Exibir o DataFrame
    print(df)

    # Salvar em um arquivo CSV para consulta futura, se necessário
    df.to_csv("transcricoes.csv", index=False, encoding="utf-8")

Processando imagem: imgs/aluno4.png
Processando imagem: imgs/aluno1.png
Processando imagem: imgs/aluno3.png
Processando imagem: imgs/aluno2.png
           arquivo                                        transcricao
0  imgs/aluno4.png  Expoente\nDELLINGA EM EDUCAÇÃO\nRUA ENDEREÇO E...
1  imgs/aluno1.png  ESTADO DE MATO GROSSO DO SUL\nSECRETARIA DE ES...
2  imgs/aluno3.png  N° Chamada: 1\nAno 1º ano\nESTADO DE MATO GROS...
3  imgs/aluno2.png  SP\nGOVERNO DO ESTADO DE SÃO PAULO\nSECRETARIA...
